# Generating dataset
---
---

# Imports

#### Standard library imports

In [1]:
import sys
import pickle
import os

sys.path.append("../../")

#### Third party imports

In [2]:
import pandas as pd
pd.set_option('display.max_columns', 500)

import numpy as np

#### Local application imports

In [3]:
%load_ext autoreload
%autoreload 2

from pkg_dir.config import *
from pkg_dir.src.utils import *
from pkg_dir.src.functions import *
from pkg_dir.src.parameters import *

# Base dataset with no metadata

### Query on database

In [ ]:
## Function parameters
db_crds = 'pc_db_prod'
query = '''
SELECT cita.citaid as appointment_id,
       citafecha as appointment_date,
       citahorad as appointment_start_time,
       citahorah as appointment_end_time,
       citaestado as appointment_status,
       citasubsecuente as recurring_patient,
       u.usuarionomfull as doctor,
       e.especialidadnom as medical_specialty,
       su.sucursalnom as clinic,
       se.servicionom as service,
       p.pacienteid as patient_id,
       p.pacientesexo as patient_sex,
       p.pacientefnac as patient_birth_date

FROM cita

 INNER JOIN usuario u ON cita.citadoctorid = u.usuarioid
 INNER JOIN servicio se ON cita.servicioid = se.servicioid
 INNER JOIN especialidad e ON u.usuarioespecialidadid = e.especialidadid
 INNER JOIN sucursal su ON cita.citasucursalid = su.sucursalid
 LEFT JOIN paciente p ON cita.pacienteid = p.pacienteid

WHERE citafecha >= '2020-01-01'
  AND citafecha <= '2023-12-31'

;
'''


In [ ]:
dfx = sql_string_to_df(db_crds, query)
dfx.columns = [
    'appointment_id', 
    'appointment_date', 
    'appointment_start_time', 
    'appointment_end_time', 
    'appointment_status', 
    'recurring_patient', 
    'doctor', 
    'medical_specialty', 
    'clinic', 
    'service', 
    'patient_id', 
    'patient_sex', 
    'patient_birth_date'
]

### Formatting date columns

In [ ]:
## Formatting appointment dates
dfx['appointment_start'] = dfx['appointment_date'].astype('str') + ' ' + dfx['appointment_start_time']
dfx['appointment_start'] = pd.to_datetime(dfx['appointment_start'], format='%Y-%m-%d %H:%M')

dfx['appointment_end'] = dfx['appointment_date'].astype('str') + ' ' + dfx['appointment_end_time']
dfx['appointment_end'] = pd.to_datetime(dfx['appointment_end'], format='%Y-%m-%d %H:%M', errors='coerce')

In [ ]:
dfx = dfx[~dfx['appointment_end'].isna()].copy()

In [ ]:
## Appointment weekday
dfx['appointment_weekday'] = dfx['appointment_start'].dt.weekday

In [ ]:
## Appointment duration
dfx['appointment_dur_min'] = dfx['appointment_end'] - dfx['appointment_start']
dfx['appointment_dur_min'] = dfx['appointment_dur_min'].dt.total_seconds() / 60

### Age of patients

In [ ]:
## Numerical age
dfx['patient_age'] = dfx['patient_birth_date'].fillna('2025')
dfx['patient_age'] = dfx['patient_age'].apply(lambda x: str(x)[:4])
dfx['patient_age'] = 2024 - dfx['patient_age'].astype('float')
dfx['patient_age'] = dfx['patient_age'].apply(lambda x: np.nan if (x <= 0 or x >=110) else x)

### Patient sex clean up

In [ ]:
dfx['patient_sex'] = dfx['patient_sex'].replace({' ': None})

### Recurring patient field

In [ ]:
dfx['recurring_patient'] = dfx['recurring_patient'].replace({' ': None})

In [ ]:
dfx['recurring_patient'] = dfx['recurring_patient'].map(
    {
        'P': 'first',
        'S': 'recurrent',
    }
)

In [ ]:
dfx['recurring_patient'].unique()

### Appointment status simplification

In [ ]:
## Reference to simplify labels in appointment status
values_map = {

    ## Completed
    'COMPLETADA': 'completed',
    'SALA_ESPERA': 'completed',
    'CONSULTA': 'completed',
    'CONFIRMADO_PAGO': 'completed',

    ## No shows
    'NO_PRESENTO': 'no_show',
    'MENSAJE_DEJADO': 'no_show',
    'NO_CONFIRMADO': 'no_show',
    'CONFIRMADO': 'no_show',
    'LLEGA_TARDE_ESPERA': 'no_show',
    'REAGENDAR': 'no_show',
    'TRIAGE': 'no_show',
    'TRIAGE_COMPLETO': 'no_show',
    'LISTA_ESPERA': 'no_show',
    'VALIDA_DATOS': 'no_show',

    ## Unused
    'DISPONIBLE': 'unused',

    ## Blocked
    'BLOQUEADO': 'blocked',

    ## Canceled
    'CANCELA_PACIENTE': 'cancel_patient',
    'CANCELA_EMPLEADO': 'cancel_employee',
    
    
}

In [ ]:
## Mapping values according to reference
dfx['appointment_status_simplified'] = dfx['appointment_status'].map(values_map).fillna(dfx['appointment_status'])

### Formatting data

In [ ]:
## Reordering columns
rc = [
 ## Appointment features
 'appointment_id',
 'appointment_weekday',
 'appointment_start',
 'appointment_end',
 'appointment_dur_min',
 'recurring_patient',
 'appointment_status_simplified',
 'clinic',
 ## Patient info
 'patient_id',
 'patient_age',
 'patient_sex',
 ## Medical info
 'doctor',
 'medical_specialty',
]

In [ ]:
dfx = dfx.loc[:, rc].copy()

### Saving/loading data

In [ ]:
## Saving df as pickle and storing it locally
path = '../../pkg_dir/data/pickles/robs_patients'
name = 'appts_extract.pkl'

pickle.dump(
    dfx,
    open(
        os.path.join(path, name),
        'wb'
    )
)

In [ ]:
## Params
path = '../../pkg_dir/data/pickles/robs_patients'
name = 'appts_extract.pkl'


## Reading extract object saved as pickle locally
pkl_obj = path + "/" + name

with open(pkl_obj, 'rb') as obj_content:
    dfx = pickle.load(obj_content)

In [ ]:
## Params
path = '../../pkg_dir/data/pickles/robs_patients'
name = 'appts_extract.pkl'

## Reading extract object saved as pickle locally
path_name = path + "/" + name

## Saving as csv
dfx.to_csv(path_name)

# Metadata actions

### Extracting from SQL

In [ ]:
sql_string = """
SELECT cita.citaid as appointment_id,
       c.cestatusobs as metadata,
       SUBSTRING(c.cestatusobs, '^[^ ]*') as metadata_usr,
       SUBSTRING(c.cestatusobs, '\((.+?)\)') as metadata_datetime,
       SUBSTRING(c.cestatusobs, ' :(.*?)  ') as metadata_action,
       SUBSTRING(c.cestatusobs, ' :(.*)') as metadata_action_full

FROM cita

 INNER JOIN citaestatus c ON cita.citaid = c.citaid AND cita.citaanio = c.citaanio


WHERE citafecha >= '2020-01-01'
  AND citafecha <= '2023-12-31'

;
"""

In [ ]:
dfx = sql_string_to_df('pc_db_prod', sql_string)

In [ ]:
dfx

In [ ]:
dfx.rename(
    columns={
        0: "appointment_id",
        1: "metadata_full",
        2: "user",
        3: "date",
        4: "action",
        5: "action_full",
    }, 
    inplace=True
)

### Pickle save/load

In [ ]:
## Saving df as pickle and storing it locally
path = '../../pkg_dir/data/pickles/robs_patients'
name = 'metadata_raw.pkl'

pickle.dump(
    dfx,
    open(
        os.path.join(path, name),
        'wb'
    )
)

In [ ]:
## Saving df as pickle and storing it locally
path = '../../pkg_dir/data/pickles/robs_patients'
name = 'metadata_raw.pkl'


## Reading extract object saved as pickle locally
pkl_obj = path + "/" + name

with open(pkl_obj, 'rb') as obj_content:
    dfm = pickle.load(obj_content)

### Cleaning actions fields

In [ ]:
dfz = dfm.copy()

In [ ]:
## Dummy col
dfz['action_clean'] = 'irrelevant'

In [ ]:
### Confirmado
mr1 = (dfz['action_full'].str.contains('Confirmado')) | (dfz['action_full'].str.contains('Confirmada'))

dfz.loc[mr1, 'action_clean'] = 'confirmed'

In [ ]:
### Creado
mr1 = (dfz['action_full'].str.contains('Creado'))
mr2 = (dfz['action_full'].str.contains('Creada'))
mr3 = (dfz['action_full'].str.contains('Creation'))

mrs = mr1 | mr2 | mr3

dfz.loc[mrs, 'action_clean'] = 'created'

In [ ]:
### No confirmado
mr1 = dfz['action_full'].str.contains('No confirmado')
mr2 = dfz['action_full'].str.contains('No confirmada')

mrs = mr1 | mr2

dfz.loc[mrs, 'action_clean'] = 'not_confirmed'

In [ ]:
### Sala espera
mr1 = dfz['action_full'].str.contains('En sala de espera')
mr2 = dfz['action_full'].str.contains('En sala de espera ')

mrs = mr1 | mr2

dfz.loc[mrs, 'action_clean'] = 'in_waiting_room'

In [ ]:
### Completado
mr1 = dfz['action_full'].str.contains('Completado')
mr2 = dfz['action_full'].str.contains('Completada')
mrs = mr1 | mr2

dfz.loc[mrs, 'action_clean'] = 'completed'

In [ ]:
### En consulta
mr1 = dfz['action_full'].str.contains('En consulta')
mr2 = dfz['action_full'].str.contains('En consulta')

mrs = mr1 | mr2

dfz.loc[mrs, 'action_clean'] = 'with_doctor'

In [ ]:
### No se presentó
mr1 = dfz['action_full'].str.contains('No se presentó')
mr2 = dfz['action_full'].str.contains('No se presento')

mrs = mr1 | mr2

dfz.loc[mrs, 'action_clean'] = 'no_show'

In [ ]:
### Cancelada por el paciente
mr1 = dfz['action_full'].str.contains('Cancelada por el paciente')
mr2 = dfz['action_full'].str.contains(' Cancelada por el paciente')
mr3 = dfz['action_full'].str.contains('Cancelled By Patient')

mrs = mr1 | mr2 | mr3

dfz.loc[mrs, 'action_clean'] = 'canceled_by_patient'

In [ ]:
### Cancelada por empledo
mr1 = dfz['action_full'].str.contains('Cancelada por empleado')
mr2 = dfz['action_full'].str.contains('Cancelada por empleado ')
mr3 = dfz['action_full'].str.contains('Cancelada por el empleado ')
mr4 = dfz['action_full'].str.contains('Cancelada por empledo')

mrs = mr1 | mr2 | mr3 | mr4

dfz.loc[mrs, 'action_clean'] = 'canceled_by_employee'

In [ ]:
### Se actualiza la fecha/hora
mr1 = dfz['action_full'].str.contains('Se actualiza la fecha/hora')
mr2 = dfz['action_full'].str.contains('Se actualiza la fecha/hora ')

mrs = mr1 | mr2

dfz.loc[mrs, 'action_clean'] = 'update_date_hour'

In [ ]:
### Disponible
mr1 = dfz['action_full'].str.contains('Disponible')
mr2 = dfz['action_full'].str.contains(' Disponible')

mrs = mr1 | mr2

dfz.loc[mrs, 'action_clean'] = 'available'

In [ ]:
### Mensaje dejado
mr1 = dfz['action_full'].str.contains('Mensaje dejado')
mr2 = dfz['action_full'].str.contains(' Mensaje dejado')

mrs = mr1 | mr2

dfz.loc[mrs, 'action_clean'] = 'message_left'

In [ ]:
### Other status
mr1 = dfz['action_full'].str.contains('Bloqueado')
mr2 = dfz['action_full'].str.contains('Triage')
mr3 = dfz['action_full'].str.contains('Validación de datos')
mr4 = dfz['action_full'].str.contains('Lista de espera')
mr5 = dfz['action_full'].str.contains('Reagendar')
mr6 = dfz['action_full'].str.contains('Llego tarde espera espacio')
mr7 = dfz['action_full'].str.contains('notification rdv')
mr8 = dfz['action_full'] == ''

mrs = mr1 | mr2 | mr3 | mr4 | mr5 | mr6 | mr7 | mr8

dfz.loc[mrs, 'action_clean'] = 'others'

###### checks

In [ ]:
dfz[dfz['action_clean'] == 'others']

In [ ]:
dfz.loc[(dfz['action_clean'] == 'irrelevant'), ['action_full', 'action_clean']] 

In [ ]:
# dfz.loc[(dfz['action_clean'] == 'irrelevant'), 'action_full'].unique().tolist()
dfz.loc[(dfz['action_clean'] == 'irrelevant'), 'action_full'].value_counts()[:50]

### Formatting and adjusting

In [ ]:
## Dropping irrelevant columns
mr1 = ~dfz['action_clean'].isin(['irrelevant', 'others'])
rc = ['appointment_id', 'user', 'date', 'action_clean']

dfz = dfz.loc[mr1, rc].copy()

In [ ]:
## Creating temporary column to homologate inconsistencies with dates
dfz['date_tmp'] = dfz['date']

In [ ]:
## Adjusting dateformat

## first format
dfz['date'] = pd.to_datetime(dfz['date_tmp'], format='%d/%m/%Y %H:%M:%S', errors='coerce')

## Dates that didn't match first format
# dfz[dfz['date'].isna()]
# dfz[~dfz['date'].isna()]

dfz.loc[dfz['date'].isna(), 'date'] = pd.to_datetime(dfz.loc[dfz['date'].isna(), 'date_tmp'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

## Dropping temporary column
dfz.drop('date_tmp', axis=1, inplace=True)

dfz = dfz.loc[~dfz['date'].isna(), :].copy()

In [ ]:
## Sorting values 
dfz.sort_values(by=['appointment_id', 'date'], inplace=True, ascending=False)

In [ ]:
dfz['no_confirm_count'] = dfz['action_clean'].eq('not_confirmed').groupby(dfz['appointment_id']).transform('sum')

In [ ]:
## Keeping latest 
dfz.drop_duplicates(keep='first', subset=['appointment_id', 'action_clean'], inplace=True)

### Pivot

In [ ]:
dfp = dfz.pivot(index='appointment_id', columns='action_clean', values=['user', 'date', 'no_confirm_count'])

In [ ]:
dfp.columns = ['meta__' + col[1] + '_$' + col[0] for col in dfp.columns]

In [ ]:
## Dropping the no confirm counts that are redundant
nrc = [
    col
    for col
    in dfp.columns
    if ('no_confirm_count' in col) & (col != 'meta__not_confirmed_$no_confirm_count')
]
dfp.drop(nrc, axis=1, inplace=True)

In [ ]:
cols = dfp.columns.tolist()
cols.sort()

dfp = dfp.loc[:, cols].copy()

### Pickle save/load

In [ ]:
## Saving df as pickle and storing it locally
path = '../../pkg_dir/data/pickles/robs_patients'
name = 'pivot_df_v2.pkl'

pickle.dump(
    dfp,
    open(
        os.path.join(path, name),
        'wb'
    )
)

In [ ]:
## Params
path = '../../pkg_dir/data/pickles/robs_patients'
name = 'pivot_df_v2.pkl'


## Reading extract object saved as pickle locally
pkl_obj = path + "/" + name

with open(pkl_obj, 'rb') as obj_content:
    dfp = pickle.load(obj_content)

# Merging metadata + new features

### Merging

In [ ]:
dfp.reset_index(inplace = True)

In [ ]:
dataset = pd.merge(dfx, dfp, on='appointment_id', how='left')

### Creation to appointment start

In [ ]:
## Appointment duration
dataset['creation_to_start_hrs'] = dataset['appointment_start'] - dataset['meta__created_$date']
dataset['creation_to_start_hrs'] = dataset['creation_to_start_hrs'].dt.total_seconds() / (60*60)
dataset['creation_to_start_hrs'] = dataset['creation_to_start_hrs'].round(2)

### Confirmation to appointment start

In [ ]:
## Appointment duration
dataset['confirm_to_start_hrs'] = dataset['appointment_start'] - dataset['meta__confirmed_$date']
dataset['confirm_to_start_hrs'] = dataset['confirm_to_start_hrs'].dt.total_seconds() / (60*60)
dataset['confirm_to_start_hrs'] = dataset['confirm_to_start_hrs'].round(2)

In [ ]:
## Confirmation tag
dataset['patient_confirm'] = 0
dataset.loc[dataset['confirm_to_start_hrs'].notnull(), 'patient_confirm'] = 1

### Patient arrival to appointment start

In [ ]:
## Appointment duration
dataset['arrival_to_start_mins'] =  dataset['appointment_start'] - dataset['meta__in_waiting_room_$date']
dataset['arrival_to_start_mins'] = dataset['arrival_to_start_mins'].dt.total_seconds() / (60)
dataset['arrival_to_start_mins'] = dataset['arrival_to_start_mins'].round(2)

### Completed to appointment start

In [ ]:
## Appointment duration
dataset['start_to_completed_mins'] = dataset['meta__completed_$date'] - dataset['appointment_start']
dataset['start_to_completed_mins'] = dataset['start_to_completed_mins'].dt.total_seconds() / (60)
dataset['start_to_completed_mins'] = dataset['start_to_completed_mins'].round(2)

### Cancel employee to appointment start

In [ ]:
dataset['cancelemp_to_start_hrs'] = dataset['appointment_start'] - dataset['meta__canceled_by_employee_$date']
dataset['cancelemp_to_start_hrs'] = dataset['cancelemp_to_start_hrs'].dt.total_seconds() / (60*60)
dataset['cancelemp_to_start_hrs'] = dataset['cancelemp_to_start_hrs'].round(2)

### Bot cancellation

In [ ]:
dataset['chatbot_cancel'] = dataset['meta__canceled_by_employee_$user'].apply(lambda x: 'chatbot' if x == 'citas.online' else 'not_chatbot')

### Cancel patient to appointment start

In [ ]:
dataset['cancelpat_to_start_hrs'] = dataset['appointment_start'] - dataset['meta__canceled_by_patient_$date']
dataset['cancelpat_to_start_hrs'] = dataset['cancelpat_to_start_hrs'].dt.total_seconds() / (60*60)
dataset['cancelpat_to_start_hrs'] = dataset['cancelpat_to_start_hrs'].round(2)

### Unifying cancels

In [ ]:
dataset['cancel_to_start_hrs'] = dataset['cancelemp_to_start_hrs']
dataset.loc[dataset['cancelpat_to_start_hrs'].notnull(), 'cancel_to_start_hrs'] = dataset.loc[dataset['cancelpat_to_start_hrs'].notnull(), 'cancelpat_to_start_hrs']

### Source of appointment

In [ ]:
## Appointment duration
dataset['appointment_source'] = dataset['meta__created_$user'].apply(lambda x: 'online' if x == 'citas.online' else 'phone')

### Appointment moved

In [ ]:
## Appointment duration
dataset['appointment_date_update'] = 'no_update'
dataset.loc[dataset['meta__update_date_hour_$user'].notnull(), 'appointment_date_update'] = 'update'

### Confirmation message sent

In [ ]:
## message for confirmation
dataset['confirm_request_msg'] = 0
dataset.loc[dataset['meta__message_left_$user'].notnull(), 'confirm_request_msg'] = 1

### Select relevant columns

In [ ]:
dataset.columns.tolist()

In [ ]:
## Reordering columns
rc = [
 ## Appointment features
 'appointment_id',
 'appointment_weekday',
 'appointment_start',
 'appointment_dur_min',
 'recurring_patient',
 'creation_to_start_hrs',
 'confirm_request_msg',
 'patient_confirm',
 'cancel_to_start_hrs',
 'chatbot_cancel', 
 'confirm_to_start_hrs',
 'arrival_to_start_mins',
 'start_to_completed_mins',
 'clinic',
 'appointment_source',
 'appointment_date_update',
 ## Patient info
 'patient_id',
 'patient_age',
 'patient_sex',
 ## Medical info
 'doctor',
 'medical_specialty',
 ## Label
 'appointment_status_simplified',
]

In [ ]:
dataset = dataset.loc[:, rc].copy()

---
# Final appointments dataset

In [ ]:
## Saving df as pickle and storing it locally
path = '../../pkg_dir/data/pickles/robs_patients'
name = 'appts_dataset.pkl'

pickle.dump(
    dataset,
    open(
        os.path.join(path, name),
        'wb'
    )
)

In [ ]:
## Params
path = '../../pkg_dir/data/pickles/robs_patients'
name = 'appts_dataset.pkl'


## Reading extract object saved as pickle locally
pkl_obj = path + "/" + name

with open(pkl_obj, 'rb') as obj_content:
    dataset = pickle.load(obj_content)

In [ ]:
## Params
path = '../../pkg_dir/data/pickles/robs_patients'
name = 'appts_dataset.csv'


## Reading extract object saved as pickle locally
path_name = path + "/" + name

dataset.to_csv(path_name)

---
# Converting to patient dataset

In [4]:
## Params
path = '../../pkg_dir/data/pickles/robs_patients'
name = 'appts_dataset.pkl'


## Reading extract object saved as pickle locally
pkl_obj = path + "/" + name

with open(pkl_obj, 'rb') as obj_content:
    dfp = pickle.load(obj_content)

In [5]:
## Eliminating rows with no patient
mr1 = dfp['patient_id'].notnull()
mr2 = ~dfp['appointment_status_simplified'].isin(
    [
        'blocked',
        'unused',
    ]
)

mrs = mr1 & mr2

dfp = dfp.loc[mrs, :].copy()

In [ ]:
dfx = dfp.groupby(
    [
        'patient_id',
    ],
    as_index=False
).agg(
    {
        'appointment_id': 'count'
    }
)

In [ ]:
dfx['appointment_id'].value_counts(normalize=True)

In [7]:
dfp.groupby(
    [
        'patient_id',
        pd.Grouper(key='appointment_start', freq='d'),
        'appointment_status_simplified',
    ],
    # as_index=True
).agg(
    {
        'appointment_id': 'count',
        'creation_to_start_hrs': 'unique',
    }
)

appointment_id  \
patient_id appointment_start appointment_status_simplified                   
1.0        2021-07-27        cancel_employee                             2   
                             cancel_patient                              1   
2.0        2021-02-15        no_show                                     1   
3.0        2020-07-08        completed                                   1   
           2020-08-10        completed                                   1   
...                                                                    ...   
301559.0   2023-05-20        cancel_employee                             1   
305563.0   2023-07-13        completed                                   1   
           2023-08-21        completed                                   2   
           2023-10-24        completed                                   1   
308208.0   2023-08-19        cancel_patient                              1   

                                                           creation_to_start_hrs  
patient_id appointment_start appointment_status_simplified                        
1.0        2021-07-27        cancel_employee                       [-0.43, 0.52]  
                             cancel_patient                              [34.19]  
2.0        2021-02-15        no_show                                     [-0.36]  
3.0        2020-07-08        completed                                  [214.31]  
           2020-08-10        completed                                   [791.4]  
...                                                                          ...  
301559.0   2023-05-20        cancel_employee                            [114.55]  
305563.0   2023-07-13        completed                                   [25.61]  
           2023-08-21        completed                          [145.98, 145.05]  
           2023-10-24        completed                                   [94.06]  
308208.0   2023-08-19        cancel_patient                             [283.83]  

[528973 rows x 2 columns]

---
# Notes

In [ ]:
## Patient with multiple appointments in one day
dfp[dfp['patient_id'] == 305563.0]

---

---